In [1]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용
from selenium.webdriver.chrome.options import Options # Selenium header 변경

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 
import time
import pandas as pd
import numpy as np
import json
import re
import csv
import requests
import pickle
import tqdm

import sqlite3

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

options = Options()
options.headless = True

import joblib

In [ ]:
service = Service(executable_path=ChromeDriverManager().install()) 

driver = webdriver.Chrome(service=service, options=options)

driver.maximize_window()

In [ ]:
metacritic_base_url = 'https://www.metacritic.com/browse/games/score/userscore/all/all/filtered?page='

driver.get(metacritic_base_url + str(0))

time.sleep(3)

In [ ]:
game_list_page = BeautifulSoup(driver.page_source, 'html.parser')

game_anchor_list = game_list_page.select('.clamp-summary-wrap > a')[:50]

In [ ]:
for file_number, anchor in tqdm.tqdm(enumerate(game_anchor_list), total=len(game_anchor_list)):
    driver.get(f"https://www.metacritic.com{anchor.attrs['href']}/user-reviews")
    
    time.sleep(3)
    
    review_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    review_dict = {}
    
    review_dict['platform'] = review_page.select_one(".product_title .platform").get_text().strip()
    review_dict['title'] = review_page.select_one(".product_title h1").get_text()
    review_dict['publisher'] = review_page.select_one(".publisher a").get_text().strip()

    for index, review in enumerate(review_page.select_one('.user_reviews').select('div.review_content')):
        
        review = review.select_one('.review_section')
        wrapped_reviews = review.select('div.review_body span')
        
        review_text = ''
        review_score = int(review.select_one('div.review_grade div').get_text())
        
        if len(wrapped_reviews) > 2:
            review_text = wrapped_reviews[2].get_text()
            if not review_text:
                review_text = wrapped_reviews[1].get_text()
        elif wrapped_reviews:
            review_text = wrapped_reviews[0].get_text()
        else:
            continue
            
        review_dict[index] = {'score' : review_score, 'review' : review_text}
            
    joblib.dump(review_dict, f'BestReview/{file_number}.pkl')

In [ ]:
dict_check = joblib.load(f'{5}.pkl')
dict_check

In [ ]:
driver.get(metacritic_base_url + str(181))

time.sleep(3)

In [ ]:
game_list_page = BeautifulSoup(driver.page_source, 'html.parser')

game_anchor_list = game_list_page.select('.clamp-summary-wrap > a')

game_anchor_list.reverse()

In [ ]:
game_anchor_list = game_anchor_list[:50]

In [ ]:
for file_number, anchor in tqdm.tqdm(enumerate(game_anchor_list), total=len(game_anchor_list)):
    driver.get(f"https://www.metacritic.com{anchor.attrs['href']}/user-reviews")
    
    time.sleep(3)
    
    review_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    review_dict = {}
    
    review_dict['platform'] = ''
    review_dict['title'] = ''
    review_dict['publisher'] = ''
    
    try:
        review_dict['platform'] = review_page.select_one(".product_title .platform").get_text().strip()
        review_dict['title'] = review_page.select_one(".product_title h1").get_text()
        review_dict['publisher'] = review_page.select_one(".publisher a").get_text().strip()
    except:
        pass

    try:
        for index, review in enumerate(review_page.select_one('.user_reviews').select('div.review_content')):

            review = review.select_one('.review_section')
            wrapped_reviews = review.select('div.review_body span')

            review_text = ''
            review_score = int(review.select_one('div.review_grade div').get_text())

            if len(wrapped_reviews) > 2:
                review_text = wrapped_reviews[2].get_text()
                if not review_text:
                    review_text = wrapped_reviews[1].get_text()
            elif wrapped_reviews:
                review_text = wrapped_reviews[0].get_text()
            else:
                continue

            review_dict[index] = {'score' : review_score, 'review' : review_text}
    except:
        continue
            
    joblib.dump(review_dict, f'WorstReview/{file_number}.pkl')

In [ ]:
driver.close()
driver.quit()